In [1]:
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import keras

In [2]:
DIRECTORY = r'./Image-Processing/catdog-data/train'

CATEGORIES = ['cats', 'dogs']

In [33]:
data = []

for category in CATEGORIES:
    path = os.path.join(DIRECTORY, category) #train/dogs and catdog-data/train/cats
    for img in os.listdir(path):
        img_path = os.path.join(path, img) #train/dogs/dog.0.jpg
        label = CATEGORIES.index(category)
        arr = cv2.imread(img_path) #read image with colors
        #arr = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE) #read images in GrayScale 
        new_arr = cv2.resize(arr, (100, 100)) #resize image
        #plt.imshow(new_arr) 
        #break
        data.append([new_arr, label]) #Assign a label to each image---> data=[feature, label=0,1] 

In [34]:
#Label of Cats and Dogs are 0 and 1 respectively. Because:

print(CATEGORIES.index("cats")) 
print(CATEGORIES.index("dogs"))

0
1


In [35]:
# Total number of images (cats+dogs)

len(data)

23000

In [36]:
# The 10th image is a cat, because of its label:

data[10][1]

0

In [37]:
# We must randomize the images:

import random

random.shuffle(data)

In [38]:
# Let's separate out features and labels and put them in X and Y:

X = []
y = []

for features, label in data:
    X.append(features)
    y.append(label)
    
X = np.array(X)
y = np.array(y)

In [39]:
y

array([0, 0, 1, ..., 0, 1, 0])

In [40]:
# Save the variables in my local computer:

import pickle
pickle.dump(X, open('X.pkl', 'wb')) # X.pkl is a name for the saved X, wb means Write in Binary
pickle.dump(y, open('y.pkl', 'wb'))

In [41]:
# Import the saved variable:

X = pickle.load(open('X.pkl', 'rb')) # rb means Read in Binary
y = pickle.load(open('y.pkl', 'rb'))

In [42]:
# Feature Scaling: scale the pixel values 0 < default_pixel < 255

X = X/255 

In [43]:
X.shape #(image_numbers, lengh, width, number_channels)

(23000, 100, 100, 3)

In [44]:
#X = X.reshape(-1, 60, 60, 1)

In [45]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

In [46]:
# Define the forward structure
model = Sequential()

model.add(Conv2D(64, (3,3), activation = 'relu')) # 64 filters (feature detector) in this layer, each 3*3
model.add(MaxPooling2D((2,2))) # the size of maxpooling is 2*2

model.add(Conv2D(64, (3,3), activation = 'relu'))
model.add(MaxPooling2D((2,2)))

model.add(Flatten()) # Flattening the arrayes

# Dense layer: 128 hidden neurons; X.shape[1:]=(200,200,3) 
model.add(Dense(128, input_shape = X.shape[1:], activation = 'relu')) 

# Final (dense) layer: 2 neurons for representing "cat" vs. "dog" classification
model.add(Dense(2, activation = 'softmax'))

In [47]:
# Define the Backpropogation components

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [48]:
model.fit(X, y, epochs=5, validation_split=0.1)

Epoch 1/5
647/647 [==============================] - 173s 268ms/step - loss: 0.6298 - accuracy: 0.6507 - val_loss: 0.5447 - val_accuracy: 0.7209
Epoch 2/5
647/647 [==============================] - 151s 233ms/step - loss: 0.4849 - accuracy: 0.7692 - val_loss: 0.4913 - val_accuracy: 0.7617
Epoch 3/5
647/647 [==============================] - 153s 237ms/step - loss: 0.4000 - accuracy: 0.8176 - val_loss: 0.4868 - val_accuracy: 0.7704
Epoch 4/5
647/647 [==============================] - 157s 243ms/step - loss: 0.2965 - accuracy: 0.8720 - val_loss: 0.5136 - val_accuracy: 0.7752
Epoch 5/5
647/647 [==============================] - 161s 248ms/step - loss: 0.1782 - accuracy: 0.9286 - val_loss: 0.7349 - val_accuracy: 0.7657


In [68]:
#from keras.callbacks import TensorBoard
#import time

X.shape

(23000, 100, 100, 3)

In [94]:
def image(path):
    img = cv2.imread(path)
    new_arr = cv2.resize(img, (100, 100))
    new_arr = np.array(new_arr)
    new_arr = new_arr.reshape(-1, 100, 100, 1)
    return new_arr


prediction = model.predict([image('./catdog-data/test1/3.jpg')])
print(CATEGORIES[prediction.argmax()])

cats
